# Рекомендация тарифов

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Цель исследования**

Построить модель с максимально большим значением *accuracy*. 

Нужно довести долю правильных ответов по крайней мере до 0.75. 

Проверить *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Вывод**

Пропусков нет, типы данных менять не нужно, названия колонок тоже в "змеином" регистре и с нижним подчеркиванием.

## Разбейте данные на выборки

Объявим две переменные:
features (англ. «признаки») — запишем в неё признаки;
target (англ. «цель») — целевой признак.

In [ ]:
features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']

Разделим исходные данные на обучающую, валидационную и тестовую выборки.


train, valid_test = train_test_split(
    df, test_size=2/5, random_state=12345)
valid, test = train_test_split(
    valid_test, train_size=0.5, random_state=12345)

features_train = train.drop(['is_ultra'], axis = 1)
features_valid = valid_test.drop(['is_ultra'], axis = 1)
features_test = test.drop(['is_ultra'], axis = 1)
target_train = df['is_ultra']
target_valid = df['is_ultra']
target_test = df['is_ultra']

In [ ]:
features, features_test, target, target_test = train_test_split(features, target, test_size=0.2, random_state=12345, stratify=target)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.6, random_state=12345, stratify=target)

In [ ]:
print(features_train.shape, features_valid.shape, features_test.shape, target_train.shape, target_valid.shape, target_test.shape)

(1028, 4) (1543, 4) (643, 4) (1028,) (1543,) (643,)


**Вывод**

Разделили данные на обучающую, валидационную и тестовую выборки. 

## Исследуйте модели

### Дерево решений

In [1]:
best_model = None
best_result = 0
dd = dict() #словарь для пары: глубина, погрешность
for depth in range(1, 8): 
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    dd[depth] = accuracy
    print('depth:', depth, ',' , 'accuracy:' , '{:>1.3f}'.format(accuracy))
print('Лучшая модель глубиной', [key for key in dd if dd[key] == max(dd.values())])

NameError: ignored

<div class="alert alert-block alert-success">
<b>Успех:</b> Подбор парамтеров был сделан верно.
</div>

### Случайный лес 

In [ ]:
best_est = 0
best_depth = 0
best_result = 0
for est in range(1, 110, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth
            

print("Количество деревьев:", best_est, "Максимальная глубина:", depth, 'Лучший показатель accuracy:' , '{:>1.3f}'.format(best_result))

<div class="alert alert-block alert-success">
<b>Успех:</b> Хорошо, что в данном случае настраивается сразу несколько параметров.
</div>

### Логистическая регрессия

In [ ]:
model =  LogisticRegression(random_state = 12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
result = model.score(features_valid, target_valid)
print('Лучший показатель accuracy:', '{:>1.3f}'.format(result))

**Вывод**

Выяснили, что для дерева решений лучшая модель глубиной 6 и 7 со значением accuracy: 0.799

Для модели случайный лес лучшие параметры следующие: Количество деревьев: 71 Максимальная глубина: 10 Лучший показатель accuracy: 0.813

Для логистической регрессии Лучший показатель accuracy: 0.713

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что попробовала несколько разных моделей.
</div>

## Проверьте модель на тестовой выборке

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=71, max_depth=10)
model.fit(features_train, target_train) 
predictions = model.predict(features_test) 
result =  model.score(features_test, target_test)
       
print('Лучший показатель accuracy:' , '{:>1.3f}'.format(best_result))

Лучший показатель accuracy: 0.810


## (бонус) Проверьте модели на адекватность

In [ ]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy.fit(features_train, target_train)
result = dummy.score(features_test, target_test)
print('', result)

 0.6936236391912908


**Вывод**

Наша модель по качеству лучше константной модели, а значит она не бесполезна.

## Общий вывод

-Разделили данные на выборки.

-Сравнили три модели и выяснили, что для дерева решений лучшая модель глубиной 6 и 7 со значением accuracy: 0.799

-Для модели случайный лес лучшие параметры следующие: Количество деревьев: 71 Максимальная глубина: 10 Лучший показатель accuracy: 0.813

-Для логистической регрессии Лучший показатель accuracy: 0.713

-Проверили модель на тестовой выборке

-Узнали, что наша модель лучше, чем случайная, а значит, что наша модель с высокой вероятностью сможет порекомендовать подходящий тариф для каждого конкретного пользователя.